In [24]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [25]:
#we have four entry in list [car_pos,car_velocity,theta,angler_velocity]
import gym
import numpy as np
import time
import matplotlib.pyplot as plt
class Monte_carlo:
    def __init__(self,env,alpha,gamma,epsilon,numberEpisodes,numberOfBins,lowerBounds,upperBounds):
        self.env=env
        self.alpha=alpha
        self.gamma=gamma
        self.epsilon=epsilon
        self.actionNumber=env.action_space.n
        self.numberEpisodes=numberEpisodes
        self.numberOfBins=numberOfBins # 20 ~ 22
        self.lowerBounds=lowerBounds
        self.upperBounds=upperBounds
        self.sumRewardsEpisode=[]
        self.Qmatrix=np.zeros((numberOfBins[0],numberOfBins[1],numberOfBins[2],numberOfBins[3],self.actionNumber),dtype=object)
        self.cartPositionBin=np.linspace(self.lowerBounds[0],self.upperBounds[0],self.numberOfBins[0])
        self.cartVelocityBin=np.linspace(self.lowerBounds[1],self.upperBounds[1],self.numberOfBins[1])
        self.poleAngleBin=np.linspace(self.lowerBounds[2],self.upperBounds[2],self.numberOfBins[2])
        self.poleAngleVelocityBin=np.linspace(self.lowerBounds[3],self.upperBounds[3],self.numberOfBins[3])
    def returnIndexState(self,state):
        position =      state[0]
        velocity =      state[1]
        angle    =      state[2]
        angularVelocity=state[3]
        indexPosition=np.maximum(np.digitize(state[0],self.cartPositionBin)-1,0)
        indexVelocity=np.maximum(np.digitize(state[1],self.cartVelocityBin)-1,0)
        indexAngle=np.maximum(np.digitize(state[2],self.poleAngleBin)-1,0)
        indexAngularVelocity=np.maximum(np.digitize(state[3],self.poleAngleVelocityBin)-1,0)
        return tuple([indexPosition,indexVelocity,indexAngle,indexAngularVelocity])
    def selectAction(self,state,index):
        if index<500 or np.random.random() < self.epsilon:
            return self.env.action_space.sample()
        else:
            curr_index = self.returnIndexState(state)
            return np.argmax(self.Qmatrix[curr_index])
    def simulateEpisodes(self):
        
        for indexEpisode in range(self.numberEpisodes):
            
            rewardsEpisode = []
            (stateS,_) = self.env.reset()
            stateS = list(stateS)
            trajectory = []  # Store (state, action, reward) tuples for the episode
            terminalState = False

            while not terminalState:
                actionA = self.selectAction(stateS, indexEpisode)
                stateSprime, reward, terminalState, _,_ = self.env.step(actionA)
                rewardsEpisode.append(reward)
                trajectory.append((self.returnIndexState(stateS), actionA, reward))
                stateS = list(stateSprime)


              # Update Q-values at the end of the episode
            G = 0  # Total return (cumulative rewards)
            gammas = [self.gamma**i for i in range(len(trajectory))]
            for i,(state_index,action,reward) in enumerate(reversed(trajectory)):
                G = reward + gammas[len(gammas)-i-1] * G
                self.Qmatrix[state_index + (action,)] += self.alpha * (G - self.Qmatrix[state_index + (action,)])

            self.sumRewardsEpisode.append(np.sum(rewardsEpisode))
            average_reward=np.mean(self.sumRewardsEpisode[-100:])
            if (indexEpisode + 1) % 100 == 0:
                print(f"Simulating episode {indexEpisode + 1} && average of rewards from [{indexEpisode - 100 + 2},{indexEpisode + 1}] = {average_reward}")
                if average_reward>=195:
                    break

            if indexEpisode%10 == 0:self.epsilon=0.99*self.epsilon
env=gym.make('CartPole-v1')
(state,_)=env.reset()

# here define the parameters for state discretization
upperBounds=env.observation_space.high
lowerBounds=env.observation_space.low
cartVelocityMin=-3
cartVelocityMax=3
poleAngleVelocityMin=-10
poleAngleVelocityMax=10
upperBounds[1]=cartVelocityMax
upperBounds[3]=poleAngleVelocityMax
lowerBounds[1]=cartVelocityMin
lowerBounds[3]=poleAngleVelocityMin

numberOfBinsPosition=20
numberOfBinsVelocity=20
numberOfBinsAngle=20
numberOfBinsAngleVelocity=20
numberOfBins=[numberOfBinsPosition,numberOfBinsVelocity,numberOfBinsAngle,numberOfBinsAngleVelocity]

# define the parameters
alpha=0.2
gamma=1
epsilon=1
numberEpisodes=20000

# create an object
Mc=Monte_carlo(env,alpha,gamma,epsilon,numberEpisodes,numberOfBins,lowerBounds,upperBounds)
Mc.simulateEpisodes()
env.close()

Simulating episode 100 && average of rewards from [1,100] = 23.76
Simulating episode 200 && average of rewards from [101,200] = 21.26
Simulating episode 300 && average of rewards from [201,300] = 23.62
Simulating episode 400 && average of rewards from [301,400] = 20.78
Simulating episode 500 && average of rewards from [401,500] = 23.31
Simulating episode 600 && average of rewards from [501,600] = 46.69
Simulating episode 700 && average of rewards from [601,700] = 60.12
Simulating episode 800 && average of rewards from [701,800] = 53.93
Simulating episode 900 && average of rewards from [801,900] = 60.08
Simulating episode 1000 && average of rewards from [901,1000] = 66.79
Simulating episode 1100 && average of rewards from [1001,1100] = 79.82
Simulating episode 1200 && average of rewards from [1101,1200] = 82.97
Simulating episode 1300 && average of rewards from [1201,1300] = 79.4
Simulating episode 1400 && average of rewards from [1301,1400] = 91.51
Simulating episode 1500 && average of

In [26]:

import gym
import numpy as np
import time
import matplotlib.pyplot as plt
class Q_Learning:
    def __init__(self,env,alpha,gamma,epsilon,numberEpisodes,numberOfBins,lowerBounds,upperBounds):
        self.env=env
        self.alpha=alpha
        self.gamma=gamma
        self.epsilon=epsilon
        self.actionNumber=env.action_space.n
        self.numberEpisodes=numberEpisodes
        self.numberOfBins=numberOfBins # 20 ~ 22
        self.lowerBounds=lowerBounds
        self.upperBounds=upperBounds
        self.sumRewardsEpisode=[]
        self.Qmatrix=np.zeros((numberOfBins[0],numberOfBins[1],numberOfBins[2],numberOfBins[3],self.actionNumber),dtype=object)
        self.cartPositionBin=np.linspace(self.lowerBounds[0],self.upperBounds[0],self.numberOfBins[0])
        self.cartVelocityBin=np.linspace(self.lowerBounds[1],self.upperBounds[1],self.numberOfBins[1])
        self.poleAngleBin=np.linspace(self.lowerBounds[2],self.upperBounds[2],self.numberOfBins[2])
        self.poleAngleVelocityBin=np.linspace(self.lowerBounds[3],self.upperBounds[3],self.numberOfBins[3])
    def returnIndexState(self,state):
        position =      state[0]
        velocity =      state[1]
        angle    =      state[2]
        angularVelocity=state[3]
        indexPosition=np.maximum(np.digitize(state[0],self.cartPositionBin)-1,0)
        indexVelocity=np.maximum(np.digitize(state[1],self.cartVelocityBin)-1,0)
        indexAngle=np.maximum(np.digitize(state[2],self.poleAngleBin)-1,0)
        indexAngularVelocity=np.maximum(np.digitize(state[3],self.poleAngleVelocityBin)-1,0)
        return tuple([indexPosition,indexVelocity,indexAngle,indexAngularVelocity])
    def selectAction(self,state,index):
        if index<500 or np.random.random() < self.epsilon:
            return self.env.action_space.sample()
        else:
            curr_index = self.returnIndexState(state)
            return np.argmax(self.Qmatrix[curr_index])
    def simulateEpisodes(self):
        
        for indexEpisode in range(self.numberEpisodes):

            rewardsEpisode=[]
            (stateS,_)=self.env.reset()
            stateS=list(stateS)
            terminalState=False
            while not terminalState:
                # return a discretized index of the state
                stateSIndex=self.returnIndexState(stateS)

                actionA = self.selectAction(stateS,indexEpisode)


                # here we step and return the state, reward, and boolean denoting if the state is a terminal state
                # prime means that it is the next state
                (stateSprime, reward, terminalState,_,_) = self.env.step(actionA)

                rewardsEpisode.append(reward)

                stateSprime=list(stateSprime)

                stateSprimeIndex=self.returnIndexState(stateSprime)

                # return the max value, we do not need actionAprime...
                QmaxPrime=np.max(self.Qmatrix[stateSprimeIndex])

                if not terminalState:
                    error=reward+self.gamma*QmaxPrime-self.Qmatrix[stateSIndex+(actionA,)]
                    self.Qmatrix[stateSIndex+(actionA,)]=self.Qmatrix[stateSIndex+(actionA,)]+self.alpha*error
                else:
                    error=reward-self.Qmatrix[stateSIndex+(actionA,)]
                    self.Qmatrix[stateSIndex+(actionA,)]=self.Qmatrix[stateSIndex+(actionA,)]+self.alpha*error
                stateS=stateSprime

            self.sumRewardsEpisode.append(np.sum(rewardsEpisode))    
            average_reward=np.mean(self.sumRewardsEpisode[-100:])
            if (indexEpisode + 1) % 100 == 0:
                print(f"Simulating episode {indexEpisode + 1} && average of rewards from [{indexEpisode - 100 + 2},{indexEpisode + 1}] = {average_reward}")
                if average_reward>=195:
                    break
            if indexEpisode%10 == 0:self.epsilon=0.99*self.epsilon
env=gym.make('CartPole-v1')
(state,_)=env.reset()

# here define the parameters for state discretization
upperBounds=env.observation_space.high
lowerBounds=env.observation_space.low
cartVelocityMin=-3
cartVelocityMax=3
poleAngleVelocityMin=-10
poleAngleVelocityMax=10
upperBounds[1]=cartVelocityMax
upperBounds[3]=poleAngleVelocityMax
lowerBounds[1]=cartVelocityMin
lowerBounds[3]=poleAngleVelocityMin

numberOfBinsPosition=20
numberOfBinsVelocity=20
numberOfBinsAngle=20
numberOfBinsAngleVelocity=20
numberOfBins=[numberOfBinsPosition,numberOfBinsVelocity,numberOfBinsAngle,numberOfBinsAngleVelocity]

# define the parameters
alpha=0.2
gamma=1
epsilon=1
numberEpisodes=20000

# create an object
Q=Q_Learning(env,alpha,gamma,epsilon,numberEpisodes,numberOfBins,lowerBounds,upperBounds)
Q.simulateEpisodes()
env.close()

Simulating episode 100 && average of rewards from [1,100] = 24.08
Simulating episode 200 && average of rewards from [101,200] = 24.99
Simulating episode 300 && average of rewards from [201,300] = 21.17
Simulating episode 400 && average of rewards from [301,400] = 21.46
Simulating episode 500 && average of rewards from [401,500] = 21.94
Simulating episode 600 && average of rewards from [501,600] = 53.18
Simulating episode 700 && average of rewards from [601,700] = 62.22
Simulating episode 800 && average of rewards from [701,800] = 75.86
Simulating episode 900 && average of rewards from [801,900] = 80.4
Simulating episode 1000 && average of rewards from [901,1000] = 87.21
Simulating episode 1100 && average of rewards from [1001,1100] = 90.0
Simulating episode 1200 && average of rewards from [1101,1200] = 101.59
Simulating episode 1300 && average of rewards from [1201,1300] = 104.92
Simulating episode 1400 && average of rewards from [1301,1400] = 107.21
Simulating episode 1500 && average 

In [27]:
import gym
import numpy as np
import time
import matplotlib.pyplot as plt
class Sarsa:
    def __init__(self,env,alpha,gamma,epsilon,numberEpisodes,numberOfBins,lowerBounds,upperBounds):
        self.env=env
        self.alpha=alpha
        self.gamma=gamma
        self.epsilon=epsilon
        self.actionNumber=env.action_space.n
        self.numberEpisodes=numberEpisodes
        self.numberOfBins=numberOfBins # 20 ~ 22
        self.lowerBounds=lowerBounds
        self.upperBounds=upperBounds
        self.sumRewardsEpisode=[]
        self.Qmatrix=np.zeros((numberOfBins[0],numberOfBins[1],numberOfBins[2],numberOfBins[3],self.actionNumber),dtype=object)
        self.cartPositionBin=np.linspace(self.lowerBounds[0],self.upperBounds[0],self.numberOfBins[0])
        self.cartVelocityBin=np.linspace(self.lowerBounds[1],self.upperBounds[1],self.numberOfBins[1])
        self.poleAngleBin=np.linspace(self.lowerBounds[2],self.upperBounds[2],self.numberOfBins[2])
        self.poleAngleVelocityBin=np.linspace(self.lowerBounds[3],self.upperBounds[3],self.numberOfBins[3])
    def returnIndexState(self,state):
        position =      state[0]
        velocity =      state[1]
        angle    =      state[2]
        angularVelocity=state[3]
        indexPosition=np.maximum(np.digitize(state[0],self.cartPositionBin)-1,0)
        indexVelocity=np.maximum(np.digitize(state[1],self.cartVelocityBin)-1,0)
        indexAngle=np.maximum(np.digitize(state[2],self.poleAngleBin)-1,0)
        indexAngularVelocity=np.maximum(np.digitize(state[3],self.poleAngleVelocityBin)-1,0)
        return tuple([indexPosition,indexVelocity,indexAngle,indexAngularVelocity])
    def selectAction(self,state,index):
        if index<500 or np.random.random() < self.epsilon:
            return self.env.action_space.sample()
        else:
            curr_index = self.returnIndexState(state)
            return np.argmax(self.Qmatrix[curr_index])
    def simulateEpisodes(self):
        
        for indexEpisode in range(self.numberEpisodes):

            rewardsEpisode = []
            stateS,_ = self.env.reset()
            stateS = list(stateS)
            actionA = self.selectAction(stateS, indexEpisode)
            terminalState = False
            while not terminalState:
                
                stateSIndex = self.returnIndexState(stateS)
                stateSprime, reward, terminalState, _ ,_= self.env.step(actionA)
                rewardsEpisode.append(reward)
                stateSprime = list(stateSprime)
                stateSprimeIndex = self.returnIndexState(stateSprime)
                actionAprime = self.selectAction(stateSprime, indexEpisode)
                Qprime = self.Qmatrix[stateSprimeIndex + (actionAprime,)]

                error = reward + self.gamma * Qprime - self.Qmatrix[stateSIndex + (actionA,)]
                self.Qmatrix[stateSIndex + (actionA,)] += self.alpha * error

                stateS = stateSprime
                actionA = actionAprime
                self.sumRewardsEpisode.append(np.sum(rewardsEpisode))    
            average_reward=np.mean(self.sumRewardsEpisode[-100:])
            if (indexEpisode + 1) % 100 == 0:
                print(f"Simulating episode {indexEpisode + 1} && average of rewards from [{indexEpisode - 100 + 2},{indexEpisode + 1}] = {average_reward}")
                if average_reward>=195:
                    break
            if indexEpisode%10 == 0:self.epsilon=0.99*self.epsilon
env=gym.make('CartPole-v1')
(state,_)=env.reset()

# here define the parameters for state discretization
upperBounds=env.observation_space.high
lowerBounds=env.observation_space.low
cartVelocityMin=-3
cartVelocityMax=3
poleAngleVelocityMin=-10
poleAngleVelocityMax=10
upperBounds[1]=cartVelocityMax
upperBounds[3]=poleAngleVelocityMax
lowerBounds[1]=cartVelocityMin
lowerBounds[3]=poleAngleVelocityMin

numberOfBinsPosition=20
numberOfBinsVelocity=20
numberOfBinsAngle=20
numberOfBinsAngleVelocity=20
numberOfBins=[numberOfBinsPosition,numberOfBinsVelocity,numberOfBinsAngle,numberOfBinsAngleVelocity]

# define the parameters
alpha=0.2
gamma=1
epsilon=1
numberEpisodes=20000

# create an object
sa=Sarsa(env,alpha,gamma,epsilon,numberEpisodes,numberOfBins,lowerBounds,upperBounds)
sa.simulateEpisodes()
env.close()

Simulating episode 100 && average of rewards from [1,100] = 19.39
Simulating episode 200 && average of rewards from [101,200] = 10.34
Simulating episode 300 && average of rewards from [201,300] = 14.19
Simulating episode 400 && average of rewards from [301,400] = 14.85
Simulating episode 500 && average of rewards from [401,500] = 19.89
Simulating episode 600 && average of rewards from [501,600] = 37.75
Simulating episode 700 && average of rewards from [601,700] = 27.59
Simulating episode 800 && average of rewards from [701,800] = 37.54
Simulating episode 900 && average of rewards from [801,900] = 50.2
Simulating episode 1000 && average of rewards from [901,1000] = 52.5
Simulating episode 1100 && average of rewards from [1001,1100] = 56.35
Simulating episode 1200 && average of rewards from [1101,1200] = 38.08
Simulating episode 1300 && average of rewards from [1201,1300] = 150.5
Simulating episode 1400 && average of rewards from [1301,1400] = 129.5
Simulating episode 1500 && average of 